# Cell Segmentation using Watershed Algorithm

In this notebook, we will segment cells from a microscopy image using the watershed algorithm. This method is advantageous for separating overlapping objects.

Let's start by loading the necessary libraries.

In [ ]:
from skimage import io, filters, measure, morphology, segmentation, color
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import pandas as pd

## Step 1: Load the Image

We will load the image from the `data` folder. Make sure the image is saved as `membranes.png` in that directory.

In [ ]:
image = io.imread('data/membranes.png')
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.show()

## Step 2: Preprocess the Image

Convert the image to grayscale if necessary. Compute the local gradients using the Sobel operator to emphasize the cell boundaries.

In [ ]:
if len(image.shape) == 3:
    from skimage.color import rgb2gray
    image_gray = rgb2gray(image)
else:
    image_gray = image

gradient = filters.sobel(image_gray)
plt.imshow(gradient, cmap='gray')
plt.title('Gradient Magnitude')
plt.show()

## Step 3: Generate Markers

Create markers for the watershed algorithm. Here, we will use a simple global threshold to find the markers.

In [ ]:
markers = ndi.label(image_gray < filters.threshold_otsu(image_gray))[0]
plt.imshow(markers, cmap='nipy_spectral')
plt.title('Markers')
plt.show()

## Step 4: Apply the Watershed Algorithm

Segment the cells using the watershed algorithm. This method uses the gradient image and the markers for segmentation.

In [ ]:
labeled_image = segmentation.watershed(gradient, markers)

plt.imshow(color.label2rgb(labeled_image, image=image, bg_label=0))
plt.title('Segmented Cells')
plt.show()

## Step 5: Post-process and Analyze Segmented Objects

Remove small objects to clean the labeled image. Then, measure properties such as area of the segmented cells.

In [ ]:
cleaned_image = morphology.remove_small_objects(labeled_image, min_size=150)
properties = measure.regionprops_table(cleaned_image, properties=['label', 'area'])

df = pd.DataFrame(properties)
df

The areas of the segmented cells have been calculated. You can further use this data for more detailed analysis.